# Table of Contents
 <p><div class="lev1 toc-item"><a href="#fix-training-reaction-degeneracies" data-toc-modified-id="fix-training-reaction-degeneracies-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>fix training reaction degeneracies</a></div><div class="lev2 toc-item"><a href="#load-lib_rxn" data-toc-modified-id="load-lib_rxn-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>load lib_rxn</a></div><div class="lev2 toc-item"><a href="#identify-wrong-degeneracy-of-training-reactions" data-toc-modified-id="identify-wrong-degeneracy-of-training-reactions-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>identify wrong degeneracy of training reactions</a></div><div class="lev2 toc-item"><a href="#fix-training-degeneracy-of-families" data-toc-modified-id="fix-training-degeneracy-of-families-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>fix training degeneracy of families</a></div>

# fix training reaction degeneracies

This script loads training reactions from all families, finds their degeneracy and outputs the correct degeneracy in its place. 

In [ ]:
from rmgpy.data.rmg import RMGDatabase
from rmgpy.chemkin import saveChemkinFile, saveSpeciesDictionary
from rmgpy.rmg.model import Species
from rmgpy.molecule import Molecule
from rmgpy import settings
from IPython.display import display, HTML, Image
import numpy as np
import os
from rmgpy.data.kinetics.family import TemplateReaction
from rmgpy.exceptions import KineticsError, UndeterminableKineticsError


## load lib_rxn

In [ ]:
database = RMGDatabase()
database.load(
    path = settings['database.directory'], 
    thermoLibraries = [],
    kineticsFamilies = 'all', 
    kineticsDepositories = ['training']
)

## identify wrong degeneracy of training reactions

This section goes through all training reactions and reports when the degeneracy is incorrect.

If the 'rewrite' option is selected, the updated rates are automatically written to the database, which is useful after PR #186 is fully implemented

In [ ]:
rewrite=False

In [ ]:
for family in database.kinetics.families.values():
    print("analyzing family {}".format(family.name))
    rxn_depository = family.depositories[0]
    training_reaction_entries = rxn_depository.entries.values()
    for entry in training_reaction_entries:
        rxn = entry.item
        old_degeneracy = rxn.degeneracy
        rxn = TemplateReaction(index = rxn.index,
                              reactants = rxn.reactants,
                              products = rxn.products,
                              kinetics = rxn.kinetics,
                              reversible = rxn.reversible,
                              duplicate = rxn.duplicate,
                              degeneracy = rxn.degeneracy,
                              pairs = rxn.pairs)
        try:
            # check if the reaction is written in forward or reverse direction
            rxn.template = family.getReactionTemplateLabels(rxn)
        except UndeterminableKineticsError:
                # This rxn in written in reverse direction.
                # Currently, we don't have to deal with degeneracy of reverse templates
                continue
        try:
            rxn.degeneracy = family.calculateDegeneracy(rxn)
        except KineticsError as e:
            print(e)
            print('entry index is {}'.format(entry.index))
            display(rxn)
        assert id(rxn.degeneracy) != id(old_degeneracy)
        if rxn.degeneracy != old_degeneracy:
            display(rxn)
            print('Degeneracy should be {1} instead of {0} for reaction {2} with index {3}'.format(old_degeneracy,rxn.degeneracy,rxn,entry.index))
    if rewrite:
        family.saveDepository(depository=rxn_depository, path=os.path.join(kinetics_families_path,rxn_depository.label))
        print('finished updating degeneracy for family {}'.format(family.label))